# DATA2001 S1 2021 Practical Assignment - Bushfire Risk Analysis (Notebook)
*An analysis of neighbourhood fire risk and median income & rent.*

**Assignment Group F14 - 3**

**Eugene Ward (SID: 311193781) & Matthew Shu (SID: 500445930)**

---

## Setup: Import libraries and create functions

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import numpy as np

import requests
import json

from shapely import wkt
from shapely.geometry import Point, Polygon, MultiPolygon
from geopandas import GeoSeries, GeoDataFrame
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.api import add_constant

# Function for accessing Postgres DB (SOURCE: DATA2001 Lab materials) - Eugene uses this
def pgconnect_using_credfile(credential_filepath):
    try:
        with open(credential_filepath) as f:
            db_conn_dict = json.load(f)
        connstring = 'postgres+psycopg2://'+db_conn_dict['user']+':'+db_conn_dict['password']+'@'+db_conn_dict['host']+'/'+db_conn_dict['database']
        db = create_engine(connstring, echo=False)
        conn = db.connect()
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return db,conn

# Function for querying the PostgreSQL DB.
# Returns value and a converted dataframe (SOURCE: DATA2001 Lab materials)
def pgquery( conn, sqlcmd, args=None, silent=False ):
    """ utility function to execute some SQL query statement
    can take optional arguments to fill in (dictionary)
    will print out on screen the result set of the query
    error and transaction handling built-in """
    retdf = pd.DataFrame()
    retval = False
    try:
        if args is None:
            retdf = pd.read_sql_query(sqlcmd,conn)
        else:
            retdf = pd.read_sql_query(sqlcmd,conn,params=args)
        if silent == False:
            print(retdf.shape)
            print(retdf.to_string())
        retval = True
    except Exception as e:
        if silent == False:
            print("db read error: ")
            print(e)
    return retval,retdf

# WKT point geom creation function (SOURCE: DATA2001 Lab materials)
def create_wkt_point_element(geom,srid):
    return WKTElement(geom.wkt, srid)

# WKT polygon geom creation function (SOURCE: DATA2001 Lab materials)
# Adapted to handle conversion of empty geometries
def create_wkt_element(geom,srid):
    if (geom.geom_type == 'Polygon'):
        geom = MultiPolygon([geom])
    elif (geom.geom_type == 'GeometryCollection'):
        geom = MultiPolygon([geom])
    return WKTElement(geom.wkt, srid)

# Z-score
def z(x, avg, sd):
    return((x-avg)/sd)

# Sigmoidal function. Did not use native exponential because fails for large negative values.
def sigmoid(x):
    return(1/(1+np.exp(-x)))

# Created and add z-score column for measure
def add_z_score_column(df, **kwargs):
    column = [c for c in df.columns.tolist() if 'density' in c][0] #Finds relevant column to calculate Z-score on
    if kwargs.get('column'):
        column = kwargs.get('column')
    mean = np.mean(df[column])
    std = np.std(df[column])
    df['z_score'] = df[column].map(lambda x: z(x, mean, std))
    return df

# Generate correlation matrix
def CorrHeatmap(df, X):
    cor = df[['fire_risk_score', X]].corr() #Native pandas correlation
    sns.heatmap(cor, annot=True)
    plt.title('Correlation')
    plt.show()

# Generate scatter chart with LM line
def LinearRegGraph(reg, y_array, X_array, X):
    plt.scatter(X_array, y_array)
    y_pred = reg.predict(X_array)
    plt.plot(X_array, y_pred)
    plt.xlabel('{X}'.format(X=X))
    plt.ylabel('Fire Risk Score')
    plt.title('Fire Risk Score regressed on {X}'.format(X=X))
    plt.show()

# Generate residuals plot
def Residual(reg, y_array, X_array, X): #Condition for creating a residual plot
    y_pred = reg.predict(X_array)
    res = y_array-y_pred
    plt.scatter(y_pred, res)
    plt.xlabel('Predicted values for {X}'.format(X=X))
    plt.ylabel('Residuals')
    plt.title('Residuals for {X}'.format(X=X))
    plt.axhline(color='r')
    plt.show()

# Generate OLS details and test values
def Stats(df, y_array, X_array):
    X2 = sm.add_constant(X_array)
    mod = sm.OLS(y_array, X2)
    res = mod.fit()
    print(res.summary())
    
# Generate all correlation analyses for values returned from our database, using functions defined above
def LinearReg(y, X):
    query = """SELECT area_id, fire_risk_score, {X} FROM neighbourhoods INNER JOIN fire_risk USING(area_id);""".format(X=X)
    response, df = pgquery(conn, query)
    df = df.dropna() #Drop rows with NaN median income and monthly rent
    print(df)
    y_array = np.array(df.loc[:,y])
    X_array = np.array(df.loc[:, X]).reshape(-1,1) #Sklearn requires multidimensional
    reg = LinearRegression().fit(X_array,y_array)
    Residual(reg, y_array, X_array, X)
    LinearRegGraph(reg, y_array, X_array, X)
    Stats(df, y_array, X_array)
    CorrHeatmap(df,X)
    return reg

print("Setup successful. Libraries imported and functions ready.")

In [ ]:
# ------ Internal use MATTHEW
#For troubleshooting in the case your data isn't loading.
#os.chdir('M:\\Jupyter Notebooks\\data2001_project')
#os.getcwd()

## Task 1: Data Integration and Database Generation

The 3 provided CSV files and 2 provided shapefiles are loaded in as dataframes/geodataframes, inspected and cleaned. We are also making use of one additional dataset, retrieved from a Geoscience Australia Web Service and converted to a geodataframe.

### Loading datasets (standard dataframes)

#### *Statistical Areas*

In [ ]:
stat_areas_df = pd.read_csv('./data/StatisticalAreas.csv')
print(stat_areas_df.dtypes)

# Cleaning - remove duplicates
print(len(stat_areas_df.area_id.unique()))

# DUPLICATE ROWS in statisticalareas - confirmed
#print(stat_areas_df.area_id.value_counts())
#stat_areas_df.loc[stat_areas_df['area_id']==106]
#stat_areas_df.loc[stat_areas_df['area_id']==111]
#stat_areas_df.loc[stat_areas_df['area_id']==114]

print(len(stat_areas_df))
stat_areas_df = stat_areas_df.drop_duplicates() # DECISION: Remove duplicate rows
print(len(stat_areas_df))

stat_areas_df.head(2)

#### *Neighbourhoods*

In [ ]:
nbhd_df = pd.read_csv('./data/Neighbourhoods.csv')

#nbhd_df.area_id.value_counts() # confirmed no duplicate area_id

# Changing column names according to assignment sheet
nbhd_df = nbhd_df.rename(columns={'number_of_dwellings':'dwellings', 'number_of_businesses':'businesses', 'median_annual_household_income':'median_income'}) 
print(len(nbhd_df))
print(nbhd_df.dtypes)

# Cleaning - correcting number representations
nbhd_df['population'] = nbhd_df['population'].str.replace(',','')
nbhd_df['population'] = nbhd_df['population'].astype('float64')
nbhd_df['dwellings'] = nbhd_df['dwellings'].str.replace(',','')
nbhd_df['dwellings'] = nbhd_df['dwellings'].astype('float64')

# Note: NaN values present in neighbourhoods
# Decision for later work - interpet NaN as 0
print(len(nbhd_df[nbhd_df.isna().any(axis=1)]))
nbhd_df[nbhd_df.isna().any(axis=1)]

#### *Business Stats*

In [ ]:
busi_stat_df = pd.read_csv('./data/BusinessStats.csv')

busi_stat_df = busi_stat_df.drop(columns=['area_name']) # Not needed in our DB design
busi_stat_df = busi_stat_df.rename(columns={'accommodation_and_food_services':'accommodation_and_food'}) # Changing column naming according to assignment sheet

#busi_stat_df.area_id.value_counts() # confirmed no duplicates
print(len(busi_stat_df))
print(busi_stat_df.dtypes)

# DB DESIGN DECISION: LIMIT BUSINESS STAT OBSERVATIONS IN DB TO THE NEIGHBOURHOODS STUDIED

print(len(busi_stat_df))
busi_stat_df = busi_stat_df.loc[busi_stat_df['area_id'].isin(nbhd_df['area_id'].tolist())]
print(len(busi_stat_df))

# Note: No NAN values present in businessstats
#busi_stat_df[busi_stat_df.isna().any(axis=1)]

In [ ]:
# DECISION: cast columns as req. in each df as string, as integrity check before going into DB as non-numeric values
stat_areas_df['area_id'] = stat_areas_df['area_id'].astype('str')
print(stat_areas_df['area_id'].dtype)

nbhd_df['area_id'] = nbhd_df['area_id'].astype('str')
print(nbhd_df['area_id'].dtype)

busi_stat_df['area_id'] = busi_stat_df['area_id'].astype('str')
print(busi_stat_df['area_id'].dtype)

stat_areas_df['parent_area_id'] = stat_areas_df['parent_area_id'].astype('str')
print(stat_areas_df['parent_area_id'].dtype)

### Loading datasets (geodataframes)
#### RFS NSW Bushfire Prone Land - shapefile

In [ ]:
rfs_gdf = gpd.read_file('./data/RFSNSW_BFPL/RFSNSW_BFPL.shp')
print(rfs_gdf.crs) # Check EPSG / CRS -- 4283 = GDA94
rfs_gdf.columns = [x.lower() for x in rfs_gdf.columns] # lower case col names
print(rfs_gdf.dtypes)

# Check geometries
print(len(rfs_gdf))
rfs_gdf.geometry.type.value_counts()

# Recreate incrementing 'gid' (0 index)
rfs_gdf.insert(loc=0, column='gid', value=rfs_gdf.index)
rfs_gdf.head()

#### ABS Statistical Area 2 (2016) - shapefile

In [ ]:
sa2_gdf = gpd.read_file('./data/1270055001_sa2_2016_aust_shape/SA2_2016_AUST.shp')

print(sa2_gdf.crs) # Check EPSG / CRS -- 4283 = GDA94
sa2_gdf.columns = [x.lower() for x in sa2_gdf.columns] # lower case col names

# Recreate incrementing 'g_id' (0 index)
sa2_gdf.insert(loc=0, column='g_id', value=sa2_gdf.index)

# Changing column names according to assignment sheet
sa2_gdf=sa2_gdf.rename(columns={'sa3_code16':'sa3_code', 'sa3_name16':'sa3_name', 'sa4_code16':'sa4_code', 'sa4_name16':'sa4_name', 'gcc_code16':'gcc_code', 'gcc_name16':'gcc_name', 'ste_code16':'ste_code', 'ste_name16':'ste_name'})
sa2_gdf['sa2_main16'] = sa2_gdf['sa2_main16'].astype('str')
sa2_gdf.head(1)

In [ ]:
# Check geometries
print(len(sa2_gdf))
print(sa2_gdf.geometry.type.value_counts())
no_geoms_sa2 = sum(sa2_gdf.geometry.type.isna())
print(f'Null geometry count: {no_geoms_sa2}')

# Inspection / preliminary exploration
print("\n")
print(sa2_gdf.ste_name.value_counts()) # Confirm federal level dataset
print("\n")
print(sa2_gdf.loc[sa2_gdf.ste_name=="New South Wales"]['gcc_name'].value_counts()) # Confirm Greater Sydney GCCSA

# Process to support report writing - confirm GS vs RONSW split of provided neighbourhoods
nbhd_join_sa2 = pd.merge(nbhd_df, sa2_gdf, left_on='area_id', right_on='sa2_main16')
print('\n')
print(nbhd_join_sa2.gcc_name.value_counts()) # 312 GS and 10 RONSW
print("\nThe assignment involves devising risk scores for all 312 Greater Sydney SA2s\nand the following 10 other NSW SA2s:\n")
nbhd_join_sa2.loc[nbhd_join_sa2.gcc_name == "Rest of NSW"]['area_name'].to_list()

# DECISION: WE RETAIN THE 18 ROWS WHERE NO SPATIAL JOINS OR FUNCTIONS CAN BE PERFORMED (NULL GEOMETRIES)
# TO MAINTAIN FULL SA2 SHAPEFILE IN OUR DB

### Additional dataset: Telephone Exchanges NSW
#### Geodataframe created from a retrieved JSON from ArcGIS REST Web Service provided by Geoscience Australia
Open API - credentials are not required to run the request.

In [ ]:
# Geoscience Australia - National Telephone Exchanges ArcGIS REST Web Service (Open API - no key required)
# https://services.ga.gov.au/gis/rest/services/Telephone_Exchanges/MapServer/0
# Usage permitted under Creative Commons Attribution 4.0 International Licence

# This service requires a relational model style syntax for its parameters, 
# e.g. WHERE (STATE)='New South Wales'
# These params then need to be converted to URL encoded characters for making the endpoint GET request

# We query the active communications exchanges in New South Wales to form our additional dataset
# with parameter: format = json

query_param = "%28STATE%29%3D%27New+South+Wales%27" # (STATE)='New South Wales'
response = requests.get("https://services.ga.gov.au/gis/rest/services/Telephone_Exchanges/MapServer/0/query?where="+
                        query_param+
                        "&f=json")

assert response.status_code == 200

exchanges_json = response.json()

# We have inspected the full JSON result in previous requests and deploy the keys accordingly

print('EPSG:\n' + str(exchanges_json['spatialReference'])) # Confirmed GDA94

# Convert results to pandas dataframe
names_recs = []
longs = []
lats = []

for i in range(0, len(exchanges_json['features'])):
    item_name = exchanges_json['features'][i]['attributes']['name']
    item_long = exchanges_json['features'][i]['geometry']['x']
    item_lat = exchanges_json['features'][i]['geometry']['y']
    names_recs.append(item_name)
    longs.append(item_long)
    lats.append(item_lat)

names_series = pd.Series(names_recs)
longs_series = pd.Series(longs)
lats_series = pd.Series(lats)

exchanges_df = pd.DataFrame({'name': names_series, 'longitude': longs_series,
                            'latitude': lats_series})

# Convert the df to geodataframe where longitude and latitude are combined into POINT geometries
exchanges_gdf = gpd.GeoDataFrame(exchanges_df,
                                       geometry=gpd.points_from_xy(exchanges_df.longitude, exchanges_df.latitude))

exchanges_gdf = exchanges_gdf.drop(columns=['longitude', 'latitude'])
exchanges_gdf.plot(color='white', edgecolor='red')

exchanges_gdf.tail()

In [ ]:
# According to the service, the point that appears to be in Victoria is Dareton, which is a border town (in NSW)
exchanges_df.loc[exchanges_df['latitude'] < -38]

# The coordinates for this point are likely an error but this town would not be in the analysis anyway
# As nature of the error is unresolved, the row is retained in the DB design.

---

### Connecting with Database and Creation of Tables
#### *Running connection function with credentials*

#### **ATTN MARKER - DO *NOT* RUN** cell below

In [ ]:
### ---------------ATTN MARKER - DO *NOT* RUN---------------
# Alternative Function for accessing Postgres DB (SOURCE: DATA2001 Lab materials) - Matthew
# JUST FOR MATTHEW

def pgconnect_using_credfile(credential_filepath):
    try:
        args = {
            'sslmode':'disable',
            'gssencmode':'disable'
        }
        with open(credential_filepath) as f:
            db_conn_dict = json.load(f)
        connstring = 'postgresql+psycopg2://'+db_conn_dict['user']+':'+db_conn_dict['password']+'@'+db_conn_dict['host']+'/'+db_conn_dict['database']
        db = create_engine(connstring, echo=False, connect_args=args)
        conn = db.connect()
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
        return None
    return db,conn

### CONNECT:

In [ ]:
# Connect to University server student Postgres DB. 
# This function will not work if not on campus or if not connected to VPN.

credfilepath = './data2x01_db.json' # Internal note: not tracked on Git, must be locally available. 
# Eugene's credentials JSON to be uploaded in submission

db, conn = pgconnect_using_credfile(credfilepath)

#### *Test connection and pgquery function with PostGIS (Postgres geospatial plugin) check*

In [ ]:
# Checking we have PostGIS working on our connection (SOURCE: DATA2001 Lab materials)

postgis_check = '''
SELECT PostGIS_Version();
'''

retval,retdf = pgquery(conn,postgis_check)
retdf

### Creation of Database Tables (from dataframes)

In [ ]:
# Check existing tables in Postgres DB public schema
for table in db.table_names():
    print(f'{table}\n')

#### Considering set relationships between datasets for key designations

In [ ]:
# Establishing set relationships between tables
test_df = stat_areas_df.copy() # 431 unique area_ids exist in statisticalareas
subset = set(nbhd_df.area_id.unique()) # 322 unique area_ids exist in neighbourhoods
test_df['exists'] = stat_areas_df.area_id.map(lambda x : True if x in subset else False)
print(test_df['exists'].value_counts())

# Statisticalareas contains a set of area_ids of which the set of neighbourhoods area_id values is a subset 
# (It is a PK in its own table and an FK in relation to statisticalareas)

# Note that there exists one more SA2 length ID in statisticalareas and that these 323 SA2s exist in the shapefile
sa2s_in_statareas = stat_areas_df.loc[stat_areas_df['area_id'].str.len() == 9]
sa2s_in_statareas.loc[~sa2s_in_statareas.area_id.isin(nbhd_df['area_id'].tolist())]

In [ ]:
# The 323 SA2s in statisticalareas are a subset of the national SA2 shapefile attribute sa2_main16
# A foreign key relationship from area_id in both neighbourhoods and statisticalareas 
# to sa2_main16 would be appropriate *IF* sa2_main16 were a primary key, however
# g_id has been designated PK; it is preferable that the 'area_id' meaning
# is preserved as definitive in the context where it can include other levels of statistical area (statisticalareas)

sa2s_in_statareas = stat_areas_df.loc[stat_areas_df['area_id'].str.len() == 9]
print(len(sa2s_in_statareas))

test_df2 = sa2_gdf.copy()
subset2 = set(sa2s_in_statareas.area_id.unique())
test_df2['exists'] = sa2_gdf.sa2_main16.map(lambda x : True if x in subset2 else False)
test_df2['exists'].value_counts()

# Finally, from a cleaning step which masked businessstats with neighbourhoods area_ids, 
# we know that businessstats will hold the same key relationships with statisticalareas as neighbourhoods

#### Informing varchar length limits
The max string length will be used to the character limits for attributes where we reasonably assume systematic naming in future.

In [ ]:
print('Max string lengths occurring in stat_areas_df columns:\n')
print(pd.Series({c: stat_areas_df[c].map(lambda x: len(str(x))).max() for c in stat_areas_df}).sort_values(ascending=False))

print('\nMax string lengths occurring in nbhd_df columns:\n')
print(pd.Series({c: nbhd_df[c].map(lambda x: len(str(x))).max() for c in nbhd_df}).sort_values(ascending=False))

print('\nMax string lengths occurring in busi_stat_df columns:\n')
print(pd.Series({c: busi_stat_df[c].map(lambda x: len(str(x))).max() for c in busi_stat_df}).sort_values(ascending=False))

# No attributes in RFS are appropriate for an assumed max varchar length for future entries

print('\nMax string lengths occurring in sa2_gdf columns:\n')
print(pd.Series({c: sa2_gdf[c].map(lambda x: len(str(x))).max() for c in sa2_gdf}).sort_values(ascending=False))

#### Statistical Areas

In [ ]:
# IF RUNNING AGAIN, THE DROP QUERY REQUIRES CASCADE BECAUSE OF FK RELATIONSHIPS 

conn.execute("DROP TABLE IF EXISTS statisticalareas CASCADE")
#conn.execute("DROP TABLE IF EXISTS statisticalareas")

stat_areas_create = '''CREATE TABLE statisticalareas (
                     area_id VARCHAR(9) NOT NULL,
                     area_name VARCHAR NOT NULL,
                     parent_area_id VARCHAR(5) NOT NULL,
                     CONSTRAINT statisticalareas_pkey PRIMARY KEY (area_id)
                     )'''

conn.execute(stat_areas_create)

In [ ]:
# Insert df data
stat_areas_df.to_sql('statisticalareas', con = conn, if_exists = 'append', index=False)
print('Data inserted into Table')

# Check table
a_response, a_df = pgquery(conn, """SELECT * FROM statisticalareas
LIMIT 1;""")
a_df.head()

#### Neighbourhoods

In [ ]:
conn.execute("DROP TABLE IF EXISTS neighbourhoods")

neighbourhoods_create = '''CREATE TABLE neighbourhoods (
                     area_id CHAR(9) NOT NULL,
                     area_name VARCHAR NOT NULL,
                     land_area FLOAT NOT NULL,
                     population NUMERIC,
                     dwellings NUMERIC NOT NULL,
                     businesses NUMERIC,
                     median_income NUMERIC,
                     avg_monthly_rent NUMERIC,
                     CONSTRAINT neighbourhoods_pkey PRIMARY KEY (area_id),
                     CONSTRAINT neighbourhoods_fkey1 FOREIGN KEY(area_id) REFERENCES statisticalareas(area_id)
                     )'''

# ALLOW NULL for 'population' 'businesses' avg_monthly_rent' 'median_income'

conn.execute(neighbourhoods_create)

In [ ]:
# Insert df data
nbhd_df.to_sql('neighbourhoods', con = conn, if_exists = 'append', index=False)
print('Data inserted into Table')

# Check table
a_response, a_df = pgquery(conn, """SELECT * FROM neighbourhoods
LIMIT 1;""")
a_df.head()

#### Business Stats

In [ ]:
conn.execute("DROP TABLE IF EXISTS businessstats")

business_create = '''CREATE TABLE businessstats (
                     area_id CHAR(9) NOT NULL,
                     number_of_businesses NUMERIC NOT NULL,
                     accommodation_and_food NUMERIC NOT NULL,
                     retail_trade NUMERIC NOT NULL,
                     agriculture_forestry_and_fishing NUMERIC NOT NULL,
                     health_care_and_social_assistance NUMERIC NOT NULL,
                     public_administration_and_safety NUMERIC NOT NULL,
                     transport_postal_and_warehousing NUMERIC NOT NULL,
                     CONSTRAINT businessstats_pkey PRIMARY KEY (area_id),
                     CONSTRAINT businessstats_fkey1 FOREIGN KEY(area_id) REFERENCES statisticalareas(area_id)
                     )'''

conn.execute(business_create)

In [ ]:
# Insert df data
busi_stat_df.to_sql('businessstats', con = conn, if_exists = 'append', index=False)
print('Data inserted into Table')

# Check table
a_response, a_df = pgquery(conn, """SELECT * FROM businessstats
LIMIT 1;""")
a_df.head()

### Creation of Database Tables (from geodataframes)

#### RFS NSW Bushfire Prone Land - shapefile

In [ ]:
srid = 4283
rfs_gdf['geom'] = rfs_gdf['geometry'].apply(lambda x: create_wkt_point_element(geom=x, srid=srid))
rfs_gdf = rfs_gdf.drop(columns="geometry")
rfs_gdf.head()

In [ ]:
conn.execute("DROP TABLE IF EXISTS rfsnsw_bfpl")

rfs_bushfire_create = '''CREATE TABLE rfsnsw_bfpl (
                     gid INTEGER PRIMARY KEY,
                     category CHAR(1),
                     shape_leng FLOAT,
                     shape_area FLOAT,
                     geom GEOMETRY(POINT, 4283)
                     )'''

conn.execute(rfs_bushfire_create)

In [ ]:
# Insert gdf data
srid = 4283
rfs_gdf.to_sql('rfsnsw_bfpl', conn, if_exists='append', index=False, 
                         dtype={'geom': Geometry('POINT', srid)})

In [ ]:
# Check table
a_response, a_df = pgquery(conn, """SELECT * FROM rfsnsw_bfpl
LIMIT 1;""")
a_df.head()

#### ABS Statistical Area 2 (2016) - shapefile

In [ ]:
# FILL NONE GEOMETRIES WITH EMPTY GEOMETRY OBJECTS SO THAT THEY CAN BE PERSISTED
# Note: a warning will be produced regarding meaning of 'isna()' in geopandas

# See: https://geopandas.readthedocs.io/en/latest/docs/reference/api/geopandas.GeoSeries.fillna.html

sa2_gdf['geometry'] = sa2_gdf['geometry'].fillna()
print(len(sa2_gdf.loc[sa2_gdf.geometry.isna()]))

srid = 4283

# WKT CONVERSION
sa2_gdf['geom'] = sa2_gdf['geometry'].apply(lambda x: create_wkt_element(geom=x, srid=srid))
sa2_gdf = sa2_gdf.drop(columns="geometry")

In [ ]:
conn.execute("DROP TABLE IF EXISTS sa2_2016_aust")

sa2_shape_create = '''CREATE TABLE sa2_2016_aust (
                     g_id INTEGER PRIMARY KEY,
                     sa2_main16 CHAR(9),
                     sa2_5dig16 CHAR(5),
                     sa2_name16 VARCHAR,
                     sa3_code   CHAR(5),
                     sa3_name   VARCHAR,
                     sa4_code   CHAR(3),
                     sa4_name   VARCHAR,
                     gcc_code   CHAR(5),
                     gcc_name   VARCHAR,
                     ste_code   CHAR(1),
                     ste_name   VARCHAR,
                     areasqkm16 FLOAT,
                     geom GEOMETRY(MULTIPOLYGON, 4283)
                     )'''

conn.execute(sa2_shape_create)

In [ ]:
# Insert gdf data
srid = 4283
sa2_gdf.to_sql('sa2_2016_aust', conn, if_exists='append', index=False, 
                         dtype={'geom': Geometry('MULTIPOLYGON', srid)})

In [ ]:
# Check table
a_response, a_df = pgquery(conn, """SELECT * FROM sa2_2016_aust WHERE g_id = 2308""")

### Creation of Database Table - Additional Dataset (Telephone exchanges geodataframe)

In [ ]:
# Use create_wkt_point_element function to create point geom column

srid = 4283
exchanges_gdf['geom'] = exchanges_gdf['geometry'].apply(lambda x: create_wkt_point_element(geom=x, srid=srid))
exchanges_gdf = exchanges_gdf.drop(columns="geometry")
exchanges_gdf.head()

In [ ]:
conn.execute("DROP TABLE IF EXISTS exchanges")

exchanges_create = '''CREATE TABLE exchanges (
                     name VARCHAR NOT NULL,
                     geom GEOMETRY(POINT, 4283),
                     CONSTRAINT exchanges_pkey PRIMARY KEY (name) 
                     )'''

conn.execute(exchanges_create)

In [ ]:
# Insert gdf data
srid = 4283
exchanges_gdf.to_sql('exchanges', conn, if_exists='append', index=False,
                     dtype={'geom': Geometry('POINT', srid)})

In [ ]:
# Check table
a_response, a_df = pgquery(conn, """ SELECT * FROM exchanges WHERE name = 'Glebe' """)

## Task 2: Fire Risk Analysis

### Creation of helpful indexes

The Bushfire Prone Land dataset is very large but we cannot refine it beforehand - during computation of the measure we need to make spatial comparisons for all of its entries using PostGIS on the Postgres server. The computation is intensive and slow so it is helpful to employ a spatial index by creating a GIST index on the rfsnsw_bfpl geom column.

In [ ]:
conn.execute("DROP INDEX IF EXISTS bfpl_geom_idx")

bf_index_create = '''CREATE INDEX bfpl_geom_idx
                          ON rfsnsw_bfpl
                      USING GIST (geom);'''

conn.execute(bf_index_create)

Since we are computing membership and relationships with the SA2 shapefiles, spatial queries involving sa2_2016_aust will benefit from this kind of index as well.

In [ ]:
conn.execute("DROP INDEX IF EXISTS sa2_geom_idx")

sa2_index_create = '''CREATE INDEX sa2_geom_idx
                          ON sa2_2016_aust
                      USING GIST (geom);'''

conn.execute(sa2_index_create)

Since the Exchanges spatial data is much smaller we deemed that index creation would not have much performance benefit.

### Computed Measures

#### Methodology note: area used in calculation of measures
For the following computed densities we use the sa2_2016_aust areasqkm16 column as that most accurately corresponds to area calculated based on geometry. The following query demonstrates our comparison process. 

There are data quality issues with the land_area column in neighbourhoods - note that some reflect a conversion from the sa2_2016_aust shapefile data to a smaller unit (e.g. Gosford-Springfield 16.9124 to 1691.2000) while others are the same (e.g. Goulburn Region at 9035.1221 in both). This is what prompted the manual check using ST_Area which affirmed the reliability of the sa2_2016_aust column. The difference in area in the pre-computed areasqkm16 vs the PostGIS result was not deemed significant enough to compute and store (or compute per query) area sizes, so areasqkm16 is used in each measure calculation.

In [ ]:
query='''
SELECT n.area_name, n.land_area, sa.areasqkm16, ST_Area(sa.geom::geography)/1000000 AS total_area_km2 
FROM neighbourhoods AS n
INNER JOIN sa2_2016_aust AS sa
ON n.area_id=sa.sa2_main16
LIMIT 10;
'''
response, checking_df = pgquery(conn, query)
checking_df['difference_postgis_orig'] = checking_df['total_area_km2'] - checking_df['areasqkm16']
checking_df['difference_orig_postgis'] = checking_df['areasqkm16'] - checking_df['total_area_km2']
print(max(checking_df['difference_postgis_orig']))
print(max(checking_df['difference_orig_postgis']))

#### 1. Population Density

In [ ]:
# As defined in methodology earlier, NaN value for population is interpreted as 0

query = '''
SELECT n.area_name, n.area_id, n.population, sa.areasqkm16, n.population/sa.areasqkm16 as population_density
FROM neighbourhoods AS n
INNER JOIN sa2_2016_aust AS sa
ON n.area_id=sa.sa2_main16
ORDER BY n.population/sa.areasqkm16 DESC;
'''
response, pop_df = pgquery(conn, query)

pop_df['population'] = pop_df['population'].fillna(0)
pop_df['population_density'] = pop_df['population_density'].fillna(0)

pop_df = add_z_score_column(pop_df)

#### 2. Dwelling Density

Dwelling and business density measures are to be summed before computing z-score as they will represent a single variable in the risk model

In [ ]:
query = '''
SELECT n.area_name, n.area_id, n.dwellings, sa.areasqkm16, n.dwellings/sa.areasqkm16 as dwelling_density
FROM neighbourhoods AS n
INNER JOIN sa2_2016_aust AS sa
ON n.area_id=sa.sa2_main16
ORDER BY n.dwellings/sa.areasqkm16 DESC;
'''

response, dwell_df = pgquery(conn, query)

#### 3. Business Density

In [ ]:
query='''
SELECT n.area_name, n.area_id, sa.areasqkm16, busi.number_of_businesses, 
busi.number_of_businesses/sa.areasqkm16 as business_density
FROM neighbourhoods AS n
INNER JOIN sa2_2016_aust AS sa
ON n.area_id=sa.sa2_main16
INNER JOIN businessstats AS busi
ON n.area_id=busi.area_id
ORDER BY busi.number_of_businesses/sa.areasqkm16 DESC;
'''
response, bus_df = pgquery(conn, query)

#### Combining measures 2 (dwelling density) and 3 (business density) for single z-score (final model includes single variable)

In [ ]:
bus_df_m = bus_df.drop(columns=['area_name','areasqkm16','number_of_businesses'])
dwellbusi_df = pd.merge(dwell_df,bus_df_m, left_on='area_id', right_on='area_id', how='inner')
dwellbusi_df = dwellbusi_df.drop(columns=['dwellings','areasqkm16'])
dwellbusi_df['dwell_and_bus_density'] = dwellbusi_df['dwelling_density'] + dwellbusi_df['business_density']
dwellbusi_df = dwellbusi_df.drop(columns=['dwelling_density','business_density'])
dwellbusi_df = add_z_score_column(dwellbusi_df)
dwellbusi_df

#### 4. BFPL Density

For this step a spatial join is performed in the database and then further computations are performed in pandas.

Step 1: SQL query that produces an output table showing matching shapefile containment relationship for every point, filtered for only the neighbourhood of interest SA2s.

In [ ]:
# WARNINGS PRODUCED WILL PREVENT PRINT OF SQL RESULT IN THE NOTEBOOK, HOWEVER DF WILL BE CREATED

query="""
SELECT * FROM
(
SELECT bf.gid, bf.category, bf.shape_leng, bf.shape_area, sa.sa2_main16, sa.sa2_name16
FROM rfsnsw_bfpl AS bf
    INNER JOIN sa2_2016_aust AS sa ON ST_Contains(sa.geom, bf.geom)
) AS nsw_all_bf
WHERE nsw_all_bf.sa2_main16 IN
(
SELECT sa.sa2_main16
FROM neighbourhoods AS n
    INNER JOIN sa2_2016_aust AS sa
    ON n.area_id=sa.sa2_main16 
)
"""

response, bfpl_step1_df = pgquery(conn, query)

In [ ]:
bfpl_step1_df.head()

Step 2: 
- Computing weightings for area based on vegetation category 
- and then aggregating total area per SA2 using pandas groupby

In [ ]:
# Function for weighting different vegetation categories
# Based on interpretation of language regarding relative risk levels and difference in required buffer distances 
# outlined in RFS Guideline for Councils to Bushfire Prone Area Land Mapping p. 11
# Note the category number coding does not represent risk ranking (while 1 is highest, 3 is medium, 2 is least)
# Inline function for this task based on post here: 
# https://stackoverflow.com/questions/41962022/apply-function-to-dataframe-column-element-based-on-value-in-other-column-for-sa

def category_weighter(number,condition):
    multiplier = {'1': 3, '2': 1, '3': 1.5}
    return number * multiplier[condition]

bfpl_step1_df['weighted_bfp_area'] = bfpl_step1_df.apply(lambda x: category_weighter(x['shape_area'],
                                                                                     x['category']), axis=1)

bfpl_step2_df = bfpl_step1_df.groupby(['sa2_main16', 'sa2_name16']).agg('weighted_bfp_area').sum().reset_index()
bfpl_step2_df.sort_values(by='weighted_bfp_area',ascending=False)

Step 3: 
- Using a join in pandas to reintroduce the neighbourhoods that contain 0 BFPL
- Computing final density (weighted_bfp_area total/areasqkm16), filling NaNs with 0s
- and computing z-scores.

In [ ]:
bfpl_step3_df = pop_df.copy()
bfpl_step3_df = bfpl_step3_df.drop(columns=['population','population_density','z_score'])
bfpl_step3_df = pd.merge(bfpl_step3_df, bfpl_step2_df, left_on='area_id', right_on='sa2_main16', how='left')
bfpl_step3_df = bfpl_step3_df.drop(columns=['sa2_main16','sa2_name16'])
bfpl_step3_df['bfpl_density'] = bfpl_step3_df['weighted_bfp_area']/bfpl_step3_df['areasqkm16']

bfpl_step3_df['weighted_bfp_area'] = bfpl_step3_df['weighted_bfp_area'].fillna(0)
bfpl_step3_df['bfpl_density'] = bfpl_step3_df['bfpl_density'].fillna(0)
bfpl_step3_df = add_z_score_column(bfpl_step3_df)

bfpl_density = bfpl_step3_df.copy()
bfpl_density.head(1)

#### 5. Assistive Service Density

In [ ]:
query='''
SELECT n.area_name, n.area_id, sa.areasqkm16, busi.health_care_and_social_assistance,
busi.health_care_and_social_assistance/sa.areasqkm16 as assistive_service_density
FROM neighbourhoods AS n
INNER JOIN sa2_2016_aust AS sa
ON n.area_id=sa.sa2_main16
INNER JOIN businessstats AS busi
ON n.area_id=busi.area_id
ORDER BY busi.health_care_and_social_assistance/sa.areasqkm16 DESC;
'''

response, ass_serv_df = pgquery(conn, query)
ass_serv_df = add_z_score_column(ass_serv_df)

#### 6. Exchange Density

In [ ]:
# We assume truth of Geoscience Australia communications exchanges dataset 
# i.e. NAN results from spatial subquery join == 0 exchanges in area and therefore 0 density
# CASE clause handles the columns which we ultimately pass to our df (so no NaNs dealt with at notebook end)

query='''
SELECT n.area_name, n.area_id, nsw_result.count AS exchanges_nans, sa.areasqkm16,
nsw_result.count/sa.areasqkm16 AS exchange_density_nans,
CASE
WHEN nsw_result.count IS NULL THEN 0
    ELSE nsw_result.count
END AS exchanges,
CASE
    WHEN nsw_result.count/sa.areasqkm16 IS NULL THEN 0
    ELSE nsw_result.count/sa.areasqkm16
END AS exchange_density
FROM neighbourhoods AS n
    INNER JOIN sa2_2016_aust AS sa
    ON n.area_id=sa.sa2_main16 
    LEFT JOIN
        (
            SELECT sa.sa2_main16, sa.sa2_name16, COUNT(e.geom)
            FROM sa2_2016_aust AS sa
                INNER JOIN exchanges AS e ON ST_Contains(sa.geom, e.geom)
            GROUP BY sa.sa2_main16, sa.sa2_name16
        ) AS nsw_result
ON n.area_id = nsw_result.sa2_main16
ORDER BY nsw_result.count/sa.areasqkm16;

'''

response, exchange_df = pgquery(conn, query)
exchange_df = exchange_df.drop(columns=['exchanges_nans','exchange_density_nans'])
exchange_df = add_z_score_column(exchange_df)

### Fire Risk Score

Provided model:

$$fire_risk = S(z(population_density)+z(dwelling_&_business_density)+z(bfpl_density)−z(assistive_service_density))$$

Refined model:

$$fire_risk = S(0.1*z(population_density)+0.1*z(dwelling_&_business_density)+3*z(bfpl_density)−0.5*z(assistive_service_density-0.5*z(exchange_density))$$

While significantly reweighting the model towards bushfire prone land density, we justify these weightings because the other variables, even with decreased weighting, still contribute to construct relative risk based on areas that are:
- populated
- under-serviced
- contain bushfire prone land
- contain relatively greater amounts of the higher risk vegetation categories (based on weightings done in preprocessing work)

Without refinement, the model is too sensitive to populated urban areas which have zero BFPL and should not receive high risk scores. BFPL is the only natural environmental variable in the model and the model determines a risk assesment of an environmental phenomenon so this has informed the significant weighting allocated.

Though the nominated coefficients would appear to potentially overwhelm the model with this variable, the computed fire risk score rankings are different to the ranked order of bushfire prone land density alone, in particular there is different relative positioning outside the top quintile of BFPL density neighbourhoods. For example, the Manly - Fairlight area is ranked 38th in BFPL density, while its fire risk score is ranked 47th, which demonstrates the model recognizes the mitigation of the BFPL risk by the two negative determinants effective in that area. While there is a heavy bias imparted by BFPL density it does not dominate the model at the expense of the informative value of the other determinants.

#### Creation of dataframe with all measures
This is the first step for the creation of the computed risk score table. A series of pandas joins integrates the computed measures into a single dataframe while ensuring correct matching with area_id.

In [ ]:
fire_risk_df = pop_df.copy()
fire_risk_df = fire_risk_df.drop(columns=['population','areasqkm16'])
fire_risk_df = fire_risk_df.rename(columns={"z_score": "pop_z_score"})

fire_risk_df = pd.merge(fire_risk_df, dwellbusi_df, left_on='area_id', right_on='area_id', how='inner')
fire_risk_df = fire_risk_df.drop(columns=['area_name_y'])
fire_risk_df = fire_risk_df.rename(columns={"z_score": "dwell_and_bus_z_score"})

fire_risk_df = pd.merge(fire_risk_df, bfpl_density, left_on='area_id', right_on='area_id', how='inner')
fire_risk_df = fire_risk_df.drop(columns=['area_name','areasqkm16','weighted_bfp_area'])
fire_risk_df = fire_risk_df.rename(columns={"z_score": "bf_z_score"})

fire_risk_df = pd.merge(fire_risk_df, ass_serv_df, left_on='area_id', right_on='area_id', how='inner')
fire_risk_df = fire_risk_df.drop(columns=['area_name','areasqkm16','health_care_and_social_assistance'])
fire_risk_df = fire_risk_df.rename(columns={"z_score": "asst_z_score"})

fire_risk_df = pd.merge(fire_risk_df, exchange_df, left_on='area_id', right_on='area_id', how='inner')
fire_risk_df = fire_risk_df.drop(columns=['area_name','areasqkm16','exchanges'])
fire_risk_df = fire_risk_df.rename(columns={"z_score": "exch_z_score"})

fire_risk_df = fire_risk_df.rename(columns={'area_name_x': 'area_name'})

fire_risk_df

#### Calculation of fire risk scores

In [ ]:
# Nominated coefficients to address confounding variable of population and the relative importance of BFPL
# 0.1(population_density)
# 0.1(dwelling_&_business_density)
# 3(bfpl_density)
# 0.5(assistive_service_density)
# 0.5(exchange_density)

fire_risk_df['untransformed'] = 0.1*(fire_risk_df['pop_z_score']) + 0.1*(fire_risk_df['dwell_and_bus_z_score']) + 3*(fire_risk_df['bf_z_score']) - 0.5*(fire_risk_df['asst_z_score']) - 0.5*(fire_risk_df['exch_z_score'])
fire_risk_df['fire_risk_score'] = fire_risk_df['untransformed'].apply(lambda x: sigmoid(x))
fire_risk_df.sort_values(by='fire_risk_score',ascending=False)

#### Integration of the computed measures and risk scores into the Postgres database

Note that the table for computed fire risk includes the computed measures per neighbourhood (direct, not z-score) and the final fire risk score per neighbourhood (which is the result of computations done in Python code, which involved use of z-scores).

In [ ]:
# Reduce the dataframe to: measures and risk score and normalised design (no 'area_name')
fire_risk_df = fire_risk_df.drop(columns=['area_name','pop_z_score','dwell_and_bus_z_score','bf_z_score','asst_z_score','exch_z_score','untransformed'])

# Table creation query
conn.execute("DROP TABLE IF EXISTS fire_risk")

score_create = '''CREATE TABLE fire_risk (
                     area_id CHAR(9) NOT NULL,
                     population_density FLOAT NOT NULL,
                     dwell_and_bus_density FLOAT NOT NULL,
                     bfpl_density FLOAT NOT NULL,
                     assistive_service_density FLOAT NOT NULL,
                     exchange_density FLOAT NOT NULL,
                     fire_risk_score FLOAT NOT NULL,
                     CONSTRAINT fire_risk_pkey PRIMARY KEY (area_id),
                     CONSTRAINT fire_risk_fkey1 FOREIGN KEY(area_id) REFERENCES statisticalareas(area_id)
                     )'''

conn.execute(score_create)

In [ ]:
fire_risk_df.to_sql('fire_risk', con = conn, if_exists = 'append', index=False)
print('Data inserted into Table')

# Check table
a_response, a_df = pgquery(conn, """SELECT * FROM fire_risk
LIMIT 1;""")

#### Generate Choropleth Map of Risk Scores

The following code is to generate a choropleth map of the fire risk scores across the New South Wales neighbourhoods studied. This is done directly from a query to the database. As a result, the geometry needs to be converted back from the WKTElement used by PostGIS to the geometry used by geopandas. (Note that for the map figure there is an implicit dependency on a library called descartes but this is available in the University server).

If query output is produced when running the below cell, please scroll to the bottom of the cell output in order to view the choropleth map (also included in Report).

In [ ]:
# WARNINGS PRODUCED WILL PREVENT PRINT OF SQL RESULT IN THE NOTEBOOK, HOWEVER DF WILL BE CREATED

response, risk_score_df = pgquery(conn, '''
SELECT area_id, fire_risk_score, ST_AsEWKT(geom::geometry)
FROM fire_risk AS fr INNER JOIN sa2_2016_aust AS sa ON(fr.area_id = sa.sa2_main16)
''') # First extract geometry as a WKTElement in string from SQL.

print(type(risk_score_df.st_asewkt[0])) 

def str_to_geom(x): # Removes unnecessary SRID and uses shapely wkt module to convert string to shapely geometry object
    x=x.replace('SRID=4283;', '')
    print(x)
    x=wkt.loads(x)
    return x

risk_score_df['geom'] = risk_score_df['st_asewkt'].map(lambda x: str_to_geom(x))
risk_score_df=risk_score_df.drop(columns=['st_asewkt'])

print(type(risk_score_df.geom[0]))

risk_score_df = GeoDataFrame(risk_score_df, crs='EPSG:4283', geometry=risk_score_df['geom'])
risk_score_df = risk_score_df.drop(columns=['geom'])

# Choropleth code
# Adapted from https://medium.com/@m_vemuri/create-a-geographic-heat-map-of-the-city-of-toronto-in-python-cd2ae0f8be55

fig, ax = plt.subplots(1, figsize=(40, 20))
ax.set_title('Fire Risk Score across Greater Sydney Neighbourhoods (New South Wales)', fontdict={'fontsize':'20'})
ax.axis('off')
color = 'Oranges'
vmin, vmax = 0, 1
sm = plt.cm.ScalarMappable(cmap=color, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A=[]
cbar = fig.colorbar(sm)
risk_score_df.plot('fire_risk_score', cmap=color, ax=ax)

### Linear Regression & Correlation

The analysis function producing these results is performed with a live query to the relevant database attributes.

#### Analysis

For each combination of regressor i.e. median income and average monthly rent, and regressand i.e. fire risk score.

* Residuals plot is first generated to check for homescedasticity assumptions so that a linear model can be applied. 
* Ordinary-least-squares method of linear regression is applied, visualised in a scatter plot along with the predicted linear model. 
* A linear model summary which most importantly includes:
    * The r2 Score which gives us an indication of how much variance in the regressand can be explained by the regressor. 
    * The p-value which gives us an indication of the significance of the parameter. The null hypothesis is that the coefficient == 0. Therefore with a higher p-value we would retain this hypothesis and it would be evidence for no relationship. Correspondingly, a p-value below the alpha threshold would mean rejection of this hypothesis and would be statistically significant evidence of a non-zero value for the coefficient, suggesting a relationship between independent and dependent variables.
    * Pearson correlation coefficient which gives us an indication of the strength and direction of the linear relationship.

#### Fire Risk Score regressed on Median Income
Based on the following analysis it is unlikely that median income can effectively explain fire risk score.

* The residuals plot shows that the data meets the assumption of homoscedasticity and the residuals are fairly normally distributed.
* The r2 score is very low indicating that the variation in fire risk score isn't well explained by median income.
* The p value is > 0.05, supporting retention of the null hypothesis (coefficient == 0)
* The correlation value indicates a very weak negative linearity.

*(Note: due to the pgquery function output that is also produced, please scroll to the bottom of the output to see the charts inside the notebook (they are also included in Report))*

*(Note: in the University server environment we need to import statsmodels in the manner below each time the function is run - this was not required in our original notebook but this adaptation should allow the cell to run without error in the marking environment)*

In [ ]:
import statsmodels.api as sm
from statsmodels.api import add_constant

LinearReg('fire_risk_score', 'median_income')

#### Fire Risk Score regressed on Average Monthly Rent

Based on the following analysis it is likely that average monthly rent could be one factor that explains fire risk score.

* The residuals plot shows that the data meets the assumption of homoscedasticity and the residuals are fairly normally distributed.
* The r2 score is relatively low indicating that the variation in fire risk score is only partially explained by median income.
* The p value is < 0.05 (in fact too small for the OLS results summary to display beyond 0.000), allowing rejection of the null hypothesis that the coefficient == 0 and providing support for a non-zero coefficient for the variable.
* The correlation value indicates some negative linearity.

*(Note: due to the pgquery function output that is also produced, please scroll to the bottom of the output to see the charts inside the notebook (they are also included in Report))*

*(Note: in the University server environment we need to import statsmodels in the manner below each time the function is run - this was not required in our original notebook but this adaptation should allow the cell to run without error in the marking environment)*

In [ ]:
import statsmodels.api as sm
from statsmodels.api import add_constant

LinearReg('fire_risk_score', 'avg_monthly_rent')

In [ ]:
# ------------------------------------
#     REMEMBER: DISCONNECT FROM DB!
# ------------------------------------

conn.close()
db.dispose()
print("disconnected")